<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(12_7_10am).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml
!pip install parse

    100% |████████████████████████████████| 5.8MB 6.3MB/s 
  Running setup.py bdist_wheel for parse ... - done
  Stored in directory: /root/.cache/pip/wheels/ef/db/c6/18568a2cc574848f3996ac4552241fbec046b7be29feb2077d
Successfully built parse


In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 28 #limit to check whether they are relative or not
NOISE = 9

In [3]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 이유몰
2 -> 7호선
3 -> 서수연
4 -> 이지안
5 -> 대설
6 -> 한고은
7 -> 조여정
8 -> 킴스닷컴
9 -> 리퍼브매장
10 -> 화웨이
11 -> 고고싱
12 -> 박항서
13 -> 이은희
14 -> 해빙
15 -> 떠리몰
16 -> 베트남 필리핀
17 -> 아리따움
18 -> 스즈키컵
19 -> 카카오톡 PC버전
20 -> 연애의 맛


In [4]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
      
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

이유몰  and  킴스닷컴
서수연  and  이지안
서수연  and  연애의 맛
이지안  and  서수연
이지안  and  이은희
이지안  and  연애의 맛
조여정  and  한고은
킴스닷컴  and  이유몰
리퍼브매장  and  이유몰
이은희  and  이지안
이은희  and  연애의 맛
베트남 필리핀  and  박항서
베트남 필리핀  and  스즈키컵
스즈키컵  and  박항서
연애의 맛  and  서수연
연애의 맛  and  이지안


In [5]:

print('\n',label,'\n')

for i in range(20):
  print(matrix[i])

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])
 
print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 3, 5, 6, 6, 1, 1, 10, 11, 12, 3, 14, 15, 12, 17, 12, 19, 3] 

[0, 0, 2, 2, 0, 2, 2, 32, 23, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2]
[2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[2, 0, 0, 28, 0, 2, 2, 0, 0, 2, 0, 2, 18, 0, 0, 0, 0, 2, 0, 69]
[2, 0, 44, 0, 0, 2, 2, 0, 0, 2, 0, 2, 65, 0, 0, 0, 0, 2, 0, 54]
[2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[2, 0, 2, 2, 0, 0, 15, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[2, 0, 2, 2, 0, 75, 0, 0, 0, 2, 0, 2, 1, 0, 0, 0, 0, 2, 0, 2]
[52, 0, 2, 2, 0, 2, 2, 0, 20, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[32, 0, 2, 2, 0, 2, 2, 3, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 17, 0, 23, 0, 2]
[2, 0, 19, 72, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 45]
[2, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2]
[22, 0, 2, 2, 0, 2, 2, 7, 3, 2, 0, 2, 0, 0, 0, 0, 0, 2

In [6]:

#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  이유몰 | 킴스닷컴 | 리퍼브매장
2 .  서수연 | 이지안 | 이은희 | 연애의 맛
3 .  7호선
4 .  한고은 | 조여정
5 .  박항서 | 베트남 필리핀 | 스즈키컵
6 .  대설
7 .  화웨이
8 .  고고싱
9 .  해빙
10 .  떠리몰
11 .  아리따움
12 .  카카오톡 PC버전
